# Getting the three nearest PanSTARRS sources from avro packets

```Authors: Francisco Förster, Last updated: 20251008```

# Introduction

ZTF objects have a unique identifier which we call `oid`, each object has many associated alerts, one per detection, and each alert as a unique identifier `candid`.

ALeRCE does not store all the information contained in the alerts in our postgresql database. However, we provide access to the individual alerts and their contents through our API. Each alert contains information about the three nearest PanSTARRS sources, so if you access any alert from a given object you can get these objects.

In this example you can see how we get the nearest PanSTARRS objects given an `oid`.

We will use the ALeRCE client (https://alerce.readthedocs.io/en/latest/index.html, which can be installed with pip) and the requests library to interact with the API.

*It is highly recommended that you try this notebook in Google Colab using the following [link](https://colab.research.google.com/github/alercebroker/usecases/blob/master/notebooks/ALeRCE_API_PanSTARRS_galaxies.ipynb).*
This will avoid you from having to sort out library installation problems and focus on the contents of the tutorial. You can try installing the dependencies later in your own system.

In [1]:
#!pip install alerce

In [2]:
from alerce.core import Alerce

In [3]:
import requests

In [4]:
client = Alerce()

# Get any candidate id from the object id

In [5]:
oid = "ZTF19acqebwl" #"ZTF19abguqsi"

Extract the minimum id of those objects that have image stamps, i.e. they also have an associated avro file

In [6]:
candids = client.query_detections(oid=oid, format='pandas')
candid = candids.loc[candids.has_stamp].candid.min()

# Get avro info from oid and candid

In [7]:
url = "https://avro.alerce.online/get_avro_info?oid=" \
      + str(oid) + "&candid=" + str(candid)
r = requests.request(url=url, method="GET")
response = r.json() if r.status_code == 200 else {}

In [8]:
# Show dictionary keys
", ".join(list(response['candidate']))

'aimage, aimagerat, bimage, bimagerat, candid, chinr, chipsf, classtar, clrcoeff, clrcounc, clrmed, clrrms, dec, decnr, diffmaglim, distnr, distpsnr1, distpsnr2, distpsnr3, drb, drbversion, dsdiff, dsnrms, elong, exptime, fid, field, fwhm, isdiffpos, jd, jdendhist, jdendref, jdstarthist, jdstartref, magap, magapbig, magdiff, magfromlim, maggaia, maggaiabright, magnr, magpsf, magzpsci, magzpscirms, magzpsciunc, mindtoedge, nbad, ncovhist, ndethist, neargaia, neargaiabright, nframesref, nid, nmatches, nmtchps, nneg, objectidps1, objectidps2, objectidps3, pdiffimfilename, pid, programid, programpi, ra, ranr, rb, rbversion, rcid, rfid, scorr, seeratio, sgmag1, sgmag2, sgmag3, sgscore1, sgscore2, sgscore3, sharpnr, sigmagap, sigmagapbig, sigmagnr, sigmapsf, simag1, simag2, simag3, sky, srmag1, srmag2, srmag3, ssdistnr, ssmagnr, ssnamenr, ssnrms, sumrat, szmag1, szmag2, szmag3, tblid, tooflag, xpos, ypos, zpclrcov, zpmed'

# Get distance, star galaxy score and object id of the the three nearest PS1 objects

In [9]:
for i in range(3):
    for key in ['objectidps%i' % (i+1), 'sgscore%i' % (i+1), 'distpsnr%i' % (i+1)]:
        print(key, response['candidate'][key])

objectidps1 118801391366765195
sgscore1 0.026624999940395355
distpsnr1 0.44015640020370483
objectidps2 118801391381720313
sgscore2 0.9841669797897339
distpsnr2 15.294926643371582
objectidps3 118801391410508642
sgscore3 0.5
distpsnr3 19.648544311523438
